In [1]:
import json
import time
import sktime as skt
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.theta import ThetaForecaster
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error

from fbprophet.serialize import model_to_json, model_from_json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from utilities import *

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [3]:
df = load_data(path = '..\Binance_BTCUSDT_1h_edited.csv')
df = preprocess_df(df, 'date', scale = False)

8  values have been droped.


In [4]:
df = df.sort_index(ascending=False)
df = df.reset_index()

In [5]:
df.drop(['level_0', 'index'], axis=1,  inplace = True)
df

,date,open,high,low,close,Volume BTC,Volume USDT,tradecount,epoch
0,2020-08-01 23:00:00,11783.31,11820.61,11731.71,11801.17,2194.598589,25849742.13,37272.0,3.0
1,2020-08-02 00:00:00,11801.17,11857.00,11740.00,11782.63,2872.638184,33906617.41,47209.0,3.0
2,2020-08-02 01:00:00,11782.63,11942.55,11772.13,11942.55,3707.136801,44004705.94,55304.0,3.0
3,2020-08-02 02:00:00,11942.54,11965.01,11892.88,11928.58,3785.415559,45157950.45,51092.0,3.0
4,2020-08-02 03:00:00,11928.40,12000.00,11870.00,12000.00,3746.422743,44731714.59,59258.0,3.0
...,...,...,...,...,...,...,...,...,...
11403,2021-11-20 20:00:00,59430.04,59538.18,59308.09,59408.03,1135.464990,67455253.53,40691.0,3.0
11404,2021-11-20 21:00:00,59408.02,59650.00,59357.21,59592.71,1007.591700,59963374.76,38975.0,3.0
11405,2021-11-20 22:00:00,59592.71,59729.68,59504.95,59683.58,968.353460,57744824.37,42021.0,3.0
11406,2021-11-20 23:00:00,59683.58,59845.00,59480.86,59707.51,1275.101520,76121126.54,39388.0,3.0


In [6]:
df['date'] = df['date'].astype("datetime64")
df

,date,open,high,low,close,Volume BTC,Volume USDT,tradecount,epoch
0,2020-08-01 23:00:00,11783.31,11820.61,11731.71,11801.17,2194.598589,25849742.13,37272.0,3.0
1,2020-08-02 00:00:00,11801.17,11857.00,11740.00,11782.63,2872.638184,33906617.41,47209.0,3.0
2,2020-08-02 01:00:00,11782.63,11942.55,11772.13,11942.55,3707.136801,44004705.94,55304.0,3.0
3,2020-08-02 02:00:00,11942.54,11965.01,11892.88,11928.58,3785.415559,45157950.45,51092.0,3.0
4,2020-08-02 03:00:00,11928.40,12000.00,11870.00,12000.00,3746.422743,44731714.59,59258.0,3.0
...,...,...,...,...,...,...,...,...,...
11403,2021-11-20 20:00:00,59430.04,59538.18,59308.09,59408.03,1135.464990,67455253.53,40691.0,3.0
11404,2021-11-20 21:00:00,59408.02,59650.00,59357.21,59592.71,1007.591700,59963374.76,38975.0,3.0
11405,2021-11-20 22:00:00,59592.71,59729.68,59504.95,59683.58,968.353460,57744824.37,42021.0,3.0
11406,2021-11-20 23:00:00,59683.58,59845.00,59480.86,59707.51,1275.101520,76121126.54,39388.0,3.0


In [7]:
df = scale_df(df, 'date')
df

,date,open,high,low,close,Volume BTC,Volume USDT,tradecount,epoch
0,2020-08-01 23:00:00,0.031303,0.030372,0.032523,0.031609,0.049607,0.017069,0.046636,3.0
1,2020-08-02 00:00:00,0.031608,0.030990,0.032665,0.031293,0.064933,0.022389,0.059070,3.0
2,2020-08-02 01:00:00,0.031292,0.032440,0.033213,0.034018,0.083796,0.029056,0.069199,3.0
3,2020-08-02 02:00:00,0.034017,0.032821,0.035272,0.033780,0.085566,0.029818,0.063928,3.0
4,2020-08-02 03:00:00,0.033776,0.033415,0.034882,0.034997,0.084684,0.029536,0.074146,3.0
...,...,...,...,...,...,...,...,...,...
11403,2021-11-20 20:00:00,0.843155,0.839550,0.844044,0.842800,0.025666,0.044541,0.050914,3.0
11404,2021-11-20 21:00:00,0.842780,0.841446,0.844882,0.845947,0.022776,0.039594,0.048767,3.0
11405,2021-11-20 22:00:00,0.845927,0.842797,0.847402,0.847496,0.021889,0.038129,0.052578,3.0
11406,2021-11-20 23:00:00,0.847475,0.844753,0.846991,0.847903,0.028822,0.050263,0.049284,3.0


In [8]:
predicted_col = "close"
train_percentage = 75
y_train, y_test = temporal_train_test_split(df[predicted_col], train_size=train_percentage)

In [9]:
x = df.close
x

0        0.031609
1        0.031293
2        0.034018
3        0.033780
4        0.034997
           ...   
11403    0.842800
11404    0.845947
11405    0.847496
11406    0.847903
11407    0.846617
Name: close, Length: 11408, dtype: float64

In [10]:
y = df.drop(['close', 'date'], axis = 1)
y

,open,high,low,Volume BTC,Volume USDT,tradecount,epoch
0,0.031303,0.030372,0.032523,0.049607,0.017069,0.046636,3.0
1,0.031608,0.030990,0.032665,0.064933,0.022389,0.059070,3.0
2,0.031292,0.032440,0.033213,0.083796,0.029056,0.069199,3.0
3,0.034017,0.032821,0.035272,0.085566,0.029818,0.063928,3.0
4,0.033776,0.033415,0.034882,0.084684,0.029536,0.074146,3.0
...,...,...,...,...,...,...,...
11403,0.843155,0.839550,0.844044,0.025666,0.044541,0.050914,3.0
11404,0.842780,0.841446,0.844882,0.022776,0.039594,0.048767,3.0
11405,0.845927,0.842797,0.847402,0.021889,0.038129,0.052578,3.0
11406,0.847475,0.844753,0.846991,0.028822,0.050263,0.049284,3.0


In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [12]:
look_back = 4
x_train, x_test, y_train, y_test = np.array(x_train) , np.array(x_test) , np.array(y_train), np.array(y_test)

In [15]:
x_train =x_train.reshape(x_train.shape[0],x_train.shape[1] , 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 1)

IndexError: tuple index out of range

In [13]:
x_train.shape

(7643,)

In [80]:
x_train = np.reshape(x_train, (x_train.shape[7643], 4, x_train.shape[7]))
x_test = np.reshape(x_test, (x_test.shape[3765], 4, x_test.shape[7]))

IndexError: tuple index out of range

In [64]:
y_test

array([[0.70788349, 0.70913205, 0.70659632, ..., 0.08803224, 0.08583895,
        3.        ],
       [0.6735624 , 0.67365022, 0.67461692, ..., 0.06840973, 0.05556265,
        3.        ],
       [0.01821381, 0.0171301 , 0.01982663, ..., 0.00859015, 0.03014742,
        3.        ],
       ...,
       [0.40603739, 0.40455299, 0.38423101, ..., 0.21172088, 0.20444666,
        3.        ],
       [0.79828104, 0.80701968, 0.79981831, ..., 0.11876726, 0.15210096,
        3.        ],
       [0.64730823, 0.65844159, 0.64263088, ..., 0.26245358, 0.21015107,
        3.        ]])

In [81]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=100, batch_size=256, verbose=2)

Epoch 1/100


ValueError: in user code:

    C:\Users\hazem\anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\hazem\anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\hazem\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\hazem\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\hazem\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\hazem\anaconda3\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    C:\Users\h